In [5]:
import numpy as np
import pandas as pd
import scipy.sparse
import xgboost as xgb
import mix_pandas as mix
import predict as predict_mix
import db_column_name as db

%matplotlib inline
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
rcParams['figure.figsize'] = 10, 7


cn = db.ColumnName()

target_minT = pd.read_csv('./data/31286_103.csv')
mix.set_index_date(target_minT, cn.date)

X = pd.read_csv('./data/character_31286.csv')
mix.set_index_date(X, cn.date)

X = X.drop([cn.point], axis=1)
X = X[[x for x in X.columns if 'avg' in x or 
       x == cn.offset]]
    
X = mix.mean_day(X)
target_minT.index = target_minT.index.round('D')

X = X.drop([cn.offset], axis=1)

target_minT = target_minT.reindex(X.index)
target_minT = mix.clean(target_minT)
X = X.reindex(target_minT.index)
X = mix.clean(X)

print(X.shape)

(1810, 54)


In [6]:
target_minT = mix.winsorized(target_minT, cn.value, [0.1, 0.80])
X = X.reindex(target_minT.index)
print(X.shape)

(1369, 54)


In [4]:
default_params = {
    'verbosity':0,
    'max_depth': 3,
    
    'min_child_weight': 3.01,
    'subsample':0.8, 
    'colsample_bytree':0.8,
    'gamma': 9,
}
reg = xgb.XGBRegressor(**default_params)

predict = predict_mix.predict_model_split(reg, X, target_minT, cn.value, 5)

for train, test in predict:
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print()
    # test.plot(ylim=(-40, 40))

c:\users\home1\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\home1\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Train size 229
Mean squared error 0.94441
Mean absolute error 0.75858
Median absolute error 0.61030
Test size 228
Mean squared error 18.55725
Mean absolute error 3.26806
Median absolute error 2.49027

Train size 457
Mean squared error 1.57737
Mean absolute error 0.96697
Median absolute error 0.75187
Test size 228
Mean squared error 13.39991
Mean absolute error 2.70867
Median absolute error 2.17180

Train size 685
Mean squared error 2.55529
Mean absolute error 1.23715
Median absolute error 0.96999
Test size 228
Mean squared error 45.03652
Mean absolute error 4.49457
Median absolute error 2.84855

Train size 913
Mean squared error 3.68401
Mean absolute error 1.46650
Median absolute error 1.18657
Test size 228
Mean squared error 44.99581
Mean absolute error 5.42401
Median absolute error 4.75557

Train size 1141
Mean squared error 4.28352
Mean absolute error 1.57727
Median absolute error 1.24886
Test size 228
Mean squared error 15.72913
Mean absolute error 3.28434
Median absolute error 3.1

In [19]:
X_3mean = X.resample('3D').mean()
X_3mean = mix.clean(X_3mean)
target_3mean = target_minT.resample('3D').mean()
target_3mean = mix.clean(target_3mean)

params = {
    'verbosity':0,
    'max_depth': 3,
    
    'min_child_weight': 3.01,
    'gamma': 9,
}
reg_mean = xgb.XGBRegressor(**params)

predict = predict_mix.predict_model_split(reg_mean, X_3mean, target_3mean, cn.value, 5)
for train, test in predict:
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print()
    # test.plot(ylim=(-40, 40))

Train size 106
Mean squared error 0.92447
Mean absolute error 0.77227
Median absolute error 0.67464
Test size 101
Mean squared error 22.42063
Mean absolute error 3.62231
Median absolute error 2.64952

Train size 207
Mean squared error 1.24739
Mean absolute error 0.86452
Median absolute error 0.71597
Test size 101
Mean squared error 10.62601
Mean absolute error 2.62474
Median absolute error 2.30842

Train size 308
Mean squared error 1.73058
Mean absolute error 1.05730
Median absolute error 0.91793
Test size 101
Mean squared error 36.11188
Mean absolute error 3.85983
Median absolute error 2.04148

Train size 409
Mean squared error 1.86247
Mean absolute error 1.08021
Median absolute error 0.90386
Test size 101
Mean squared error 48.73883
Mean absolute error 5.56155
Median absolute error 4.35460

Train size 510
Mean squared error 1.84944
Mean absolute error 1.07234
Median absolute error 0.85230
Test size 101
Mean squared error 17.70056
Mean absolute error 3.48035
Median absolute error 3.18